In [3]:
import os
import socket
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB, GaussianNB, CategoricalNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.metrics import accuracy_score

if socket.gethostname() == 'LTSSL-sKTPpP5Xl':
    data_dir = 'C:\\Users\\ams90\\PycharmProjects\\ConceptsBirds\\data'
elif socket.gethostname() == 'LAPTOP-NA88OLS1':
    data_dir = 'D:\\data\\caltecBirds\\CUB_200_2011'
else:
    data_dir = '/home/bwc/ams90/datasets/caltecBirds/CUB_200_2011'


In [4]:
attributes       = pd.read_csv(os.path.join(data_dir, 'attributes.txt'),         sep=" ", index_col=[0], names=['attribute_id', 'attribute_name'])
train_test_split = pd.read_csv(os.path.join(data_dir, 'train_test_split.txt'),   sep=" ", index_col=[0], names=['image_id', 'trainset'])
image_file_names = pd.read_csv(os.path.join(data_dir, 'images.txt'),             sep=" ", index_col=[0], names=['image_id', 'file_name'])
class_labels     = pd.read_csv(os.path.join(data_dir, 'image_class_labels.txt'), sep=" ", index_col=[0], names=['image_id', 'class_id'])
image_attribute_labels = pd.read_csv(os.path.join(data_dir, 'attributes', 'image_attribute_labels.txt'), sep=" ", 
                                     names = ['image_id', 'attribute_id', 'present', 'certainty_id', 'time', 'd1', 'd2'])
image_attribute_labels = image_attribute_labels.merge(train_test_split, on='image_id', how='left' )
image_attribute_labels = image_attribute_labels.merge(class_labels, on='image_id', how='left' )

In [5]:
y_train, y_test = np.array(class_labels.loc[train_test_split['trainset']==1]['class_id']), np.array(class_labels.loc[train_test_split['trainset']==0]['class_id'])
X_train = np.array(image_attribute_labels.loc[image_attribute_labels['trainset']==1].copy().pivot_table(index='image_id', columns='attribute_id')['present'], dtype=np.int8)
X_test = np.array(image_attribute_labels.loc[image_attribute_labels['trainset']==0].copy().pivot_table(index='image_id', columns='attribute_id')['present'], dtype=np.int8)

In [6]:
# Naive Bayes
classifiers = {'Bernoulli': BernoulliNB(alpha=1), 'Gaussian': GaussianNB(), 'Categorical': CategoricalNB(), 'Multinomial': MultinomialNB()}

for name, classifier in classifiers.items():

    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test)

    # Evaluate the model's accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Naive Bayes {name}  Accuracy: {accuracy:.4f}")

Naive Bayes Bernoulli  Accuracy: 0.4256
Naive Bayes Gaussian  Accuracy: 0.2133
Naive Bayes Categorical  Accuracy: 0.4256
Naive Bayes Multinomial  Accuracy: 0.4342


In [7]:
# Decision Trees
classifiers = {'Entropy ': DecisionTreeClassifier(criterion='entropy'),
               'Gini    ': DecisionTreeClassifier(criterion='gini'),
               'LogLoss ': DecisionTreeClassifier(criterion='log_loss'),
               }


for name, classifier in classifiers.items():

    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test)

    # Evaluate the model's accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Decision Tree {name}  Accuracy: {accuracy:.4f}")

Decision Tree Entropy   Accuracy: 0.2090
Decision Tree Gini      Accuracy: 0.2478
Decision Tree LogLoss   Accuracy: 0.2097


In [8]:
# Ensembles
classifiers = {'Bagging ': BaggingClassifier(DecisionTreeClassifier(criterion='gini'), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1),
               'RandomForest': RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1),
               'AdaBoost':     AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=200, algorithm="SAMME", learning_rate=0.5)              
               }


for name, classifier in classifiers.items():

    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test)

    # Evaluate the model's accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Ensemble {name}  Accuracy: {accuracy:.4f}")

Ensemble Bagging   Accuracy: 0.3369
Ensemble RandomForest  Accuracy: 0.2765


C:\Users\ams90\.conda\envs\pytorch\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Ensemble AdaBoost  Accuracy: 0.2718


In [9]:
# Feature Importance
classifier = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

feature_scores = pd.DataFrame(classifier.feature_importances_, columns=['score']).merge(attributes, left_index=True, right_index=True)


In [10]:
for i, record in enumerate(feature_scores.sort_values(by='score', ascending=False).iterrows()):
    print(i+1, '\t',  record[1]['attribute_name'], '\t',record[1]['score'], )

1 	 has_crown_color::white 	 0.030280418513961092
2 	 has_throat_color::grey 	 0.027782889371695155
3 	 has_throat_color::orange 	 0.023865087974794304
4 	 has_bill_color::buff 	 0.02282601278789367
5 	 has_bill_length::shorter_than_head 	 0.02249945194443653
6 	 has_underparts_color::grey 	 0.022123414166325597
7 	 has_bill_shape::hooked 	 0.020600090453867476
8 	 has_belly_color::grey 	 0.02052035702797254
9 	 has_forehead_color::grey 	 0.020366093050827743
10 	 has_breast_color::white 	 0.019505114955861557
11 	 has_shape::long-legged-like 	 0.01894723968855374
12 	 has_primary_color::grey 	 0.01858822698748824
13 	 has_bill_shape::cone 	 0.018517419208236605
14 	 has_bill_shape::needle 	 0.018056019582614994
15 	 has_forehead_color::white 	 0.017835915235132074
16 	 has_breast_color::grey 	 0.01774177827565828
17 	 has_bill_shape::all-purpose 	 0.017645457716120166
18 	 has_nape_color::white 	 0.017248878677831217
19 	 has_bill_length::longer_than_head 	 0.016756155653466442
20 	 h